### 请先查看README.md，生成ubt_data

In [10]:
import pickle
import networkx as nx

with open("ubt_data/modules_info.pkl", "rb") as f:
    modules_info = pickle.load(f)

In [11]:
modules_info['OpenColorIO']['Directory']

'D:\\UnrealEngine\\Engine\\Plugins\\Compositing\\OpenColorIO\\Source\\OpenColorIO'

In [12]:
import os
import shutil
import logging
import json
import uuid
import matplotlib.pyplot as plt

# Info logger that logs to a file
info_logger = logging.getLogger('info_logger')
info_logger.setLevel(logging.INFO)
info_handler = logging.FileHandler('logs/filesystem.log')
info_handler.setFormatter(logging.Formatter('%(asctime)s - %(levelname)s - %(message)s'))
info_logger.addHandler(info_handler)

# Error logger that logs both to console and a file
error_logger = logging.getLogger('error_logger')
error_logger.setLevel(logging.ERROR)
error_handler = logging.FileHandler('logs/error.log')
error_handler.setFormatter(logging.Formatter('%(asctime)s - %(levelname)s - %(message)s'))
error_logger.addHandler(error_handler)
console_handler = logging.StreamHandler()
console_handler.setFormatter(logging.Formatter('%(asctime)s - %(levelname)s - %(message)s'))
error_logger.addHandler(console_handler)

class FileSystem:

    def __init__(self, trash_dir='trash', trash_log_file='logs/trash_log.json'):
        self.trash_dir = trash_dir
        self.trash_log_file = trash_log_file

        if os.path.exists(trash_log_file): 
            with open(trash_log_file, 'r') as f:
                self.trash = json.load(f)
        else:
            self.trash = {}

    def delete(self, nodes):
        for node in nodes:
            try:
                dir_path = modules_info[node]["Directory"]
            except KeyError:
                error_logger.error(f'No directory found for node: {node}')
                continue 
            if os.path.exists(dir_path):
                unique_name = f"{os.path.basename(dir_path)}_{uuid.uuid4().hex}"
                trash_path = os.path.join(self.trash_dir, unique_name)
                shutil.move(dir_path, trash_path)
                self.trash[node] = {
                    "dir_path": dir_path,
                    "trash_path": trash_path,
                    "original_name": os.path.basename(dir_path)
                }
                info_logger.info(f'Deleted directory: {dir_path}, moved to: {trash_path}')
        self.save_trash_log()

    def restore(self):
        if not self.trash:
            info_logger.info('No directories to restore.')
            return

        # Restore the latest directory.
        node, paths = list(self.trash.items())[-1]
        self.restore_specific(node)

    def restore_specific(self, node):
        paths_info = self.trash.get(node)
        if not paths_info:
            error_logger.error(f'No directory info found for node: {node}')
            return

        dir_path = paths_info["dir_path"]
        trash_path = paths_info["trash_path"]

        # Check if dir_path already exists
        if os.path.exists(dir_path):
            error_logger.error(f'Target directory already exists: {dir_path}. Restoration skipped for node: {node}')
            return

        if os.path.exists(trash_path):
            try:
                # 在恢复时使用原始名称
                original_name = self.trash[node]["original_name"]
                restore_path = os.path.join(os.path.dirname(dir_path), original_name)
                shutil.move(trash_path, restore_path)
                info_logger.info(f'Restored directory: {dir_path} from: {trash_path}')
                del self.trash[node]
                self.save_trash_log()
            except shutil.Error as e:
                error_logger.error(f'Error restoring directory: {e}')
    
    def check_existence_and_print(self):
        existing_dirs = {}
        for node, paths_info in self.trash.items():
            dir_path = paths_info["dir_path"]
            if os.path.exists(dir_path):
                print(f"Node: {node}, Dir Path: {dir_path}")
                existing_dirs[node] = dir_path
        return existing_dirs
    
    def save_trash_log(self):
        with open(self.trash_log_file, 'w') as f:
            json.dump(self.trash, f)
            
fs = FileSystem()

In [13]:
fs.delete(["XMPP"])

In [14]:
fs.restore_specific('XMPP')